Importing packages for face recognition

In [1]:
import os,sys
from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np 

Preparing gender dataset

In [2]:
male_list = []
male_txt = open('./gender/male.txt', 'r')
lines = male_txt.readlines()
for line in lines:
    line = line.replace('\n', '')
    male_list.append(line)
male_txt.close()

female_list = []
female_txt = open('./gender/female.txt', 'r')
lines = female_txt.readlines()
for line in lines:
    line = line.replace('\n', '')
    female_list.append(line)
female_txt.close()

Extracting sift features and preparing train/test dataset

In [3]:
# #############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(data_home = 'D:\\dataset', min_faces_per_person=5)

n_samples, h, w = lfw_people.images.shape

X = []
labels = []
for i in range(n_samples):

    face_image = lfw_people.images[i]
    name = lfw_people.target_names[lfw_people.target[i]]
    name = name.replace('-', ' ')
    name = name.replace('_', ' ')
    if name in male_list:
        id = 0
    elif name in female_list:
        id = 1
    labels.append(id)

# X = np.reshape(X,(n_samples,128))
labels = np.reshape(labels,(n_samples,))

X = lfw_people.data

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("n_samples: %d" % n_samples)
print("n_classes: ", 2)


n_samples: 5985
n_classes:  2


Train and Test

In [4]:
# #############################################################################
# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.25, random_state=42
)

# #############################################################################
# Train a SVM classification model

print("Fitting the classifier to the training set")
t0 = time()
param_grid = {
    "C": [1e3, 5e3, 1e4, 5e4, 1e5],
    "gamma": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
clf = GridSearchCV(SVC(kernel="rbf", class_weight="balanced"), param_grid)
clf = clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)



score = clf.score(X_test, y_test)
print('Score:', score)

Fitting the classifier to the training set


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


done in 2940.446s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Score: 0.8289913159652639


Saving the model

In [5]:
import pickle

Pkl_Filename = "gender-svm-{0:.4f}.pkl".format(score)
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clf, file)